Bonhoure Timothé et Martinez Christophe

# GNN : Projet

In [1]:
#import

import torch
from torch_geometric.utils import from_networkx, train_test_split_edges
import networkx as nx
from sklearn.preprocessing import LabelEncoder

from GCN import GCN, SimpleGCN

from sklearn.metrics import confusion_matrix
import seaborn as sns

import pandas as pd
from matplotlib import pyplot as plt

In [2]:
# Load the network
G = nx.read_graphml("data/airportsAndCoordAndPop.graphml")

In [ ]:
node_positions = {node: (data['lon'], data['lat']) for node, data in G.nodes(data=True)}
labels = {node: (data['city_name']) for node, data in G.nodes(data=True)}
nx.draw_networkx_edges
nx.draw(G, pos=node_positions, with_labels=True, labels=labels, font_size=1, node_size=10, width = 0.1)
plt.plot()

In [3]:
G.graph = {} 
data = from_networkx(G, group_node_attrs=["lon","lat"])

print(data)
print(data.num_features)
print(len(set(data.country)))

Data(edge_index=[2, 27094], population=[3363], country=[3363], city_name=[3363], x=[3363, 2])
2
212


In [4]:
encoder = LabelEncoder()
integer_labels = encoder.fit_transform(data.country)
target_tensor = torch.tensor(integer_labels, dtype=torch.long)
data.y = target_tensor
data.num_classes = len(set(data.country))

In [5]:
num_nodes = data.num_nodes
train_ratio = 0.80 # 80% of nodes for training
# Randomly creating a mask
mask = torch.rand(num_nodes) < train_ratio
data.train_mask = mask
data.test_mask = data.train_mask
# remove the attributes for the nodes that are not in the training set
temp = torch.zeros((num_nodes, 2), dtype=torch.float)
temp[data.train_mask] = data.x[data.train_mask]
data.x = temp

In [6]:
print(data.x)

tensor([[   0.0000,    0.0000],
        [   0.0000,    0.0000],
        [-149.6000,  -17.5500],
        ...,
        [-113.2039,   54.7431],
        [ 146.6000,   -6.1333],
        [ 147.2500,   -6.0000]])


In [7]:
model = GCN(dim_in=data.num_features, dim_h=100, dim_out=data.num_classes)
model.fit(data, 1000)

In [8]:
pred = model(data.x, data.edge_index).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f"Accuracy: {acc:.4f}")

Accuracy: 0.7467


In [34]:
G.graph = {} 
encoder = LabelEncoder()
encoder.fit([G.nodes[node]["country"] for node in G.nodes])

for node in G.nodes :
    G.nodes[node]["country_code"] = encoder.transform([G.nodes[node]["country"]])[0]


data = from_networkx(G, group_node_attrs=["lon","lat","country_code"])

print(data)
print(data.num_features)

Data(edge_index=[2, 27094], population=[3363], country=[3363], city_name=[3363], x=[3363, 3])
3


In [59]:
from encoder import Encoder

dt = data.__copy__()
train_test_split_edges(dt)
print(dt)
dt.num_classes = len(set(data.country))

encoder = Encoder(in_channels=dt.num_features, out_channels=15)

Data(population=[3363], country=[3363], city_name=[3363], x=[3363, 3], val_pos_edge_index=[2, 677], test_pos_edge_index=[2, 1354], train_pos_edge_index=[2, 23032], train_neg_adj_mask=[3363, 3363], val_neg_edge_index=[2, 677], test_neg_edge_index=[2, 1354])


/Users/timothebonhoure/miniconda3/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [60]:
from torch_geometric.nn import VGAE

vgae = VGAE(encoder)

def fit(model, data, epochs):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    model.train()
    for epoch in range(epochs+1):
        if epoch % 100 == 0 :
            z = vgae.encode(dt.x, dt.train_pos_edge_index)
            print(epoch, " : ", vgae.test(z, dt.test_pos_edge_index, dt.test_neg_edge_index))
        optimizer.zero_grad()
        z = model.encode(dt.x, dt.train_pos_edge_index)
        loss = model.recon_loss(z, data.train_pos_edge_index) + (1 / dt.num_nodes) * model.kl_loss()
        loss.backward()
        optimizer.step()

fit(vgae, dt, 2000)

z = vgae.encode(dt.x, dt.train_pos_edge_index)
vgae.test(z, dt.test_pos_edge_index, dt.test_neg_edge_index)

0  :  (0.4881831610044313, 0.4942293862047879)
100  :  (0.8305354886991658, 0.7571668982627946)
200  :  (0.8743260845375266, 0.8104384973279698)
300  :  (0.9203688834876257, 0.8855916985121329)
400  :  (0.9318295372974436, 0.9382725169564655)
500  :  (0.9440783803774145, 0.949852914119183)
600  :  (0.948329147839216, 0.9532897754640356)
700  :  (0.9559170377610844, 0.9582607429820684)
800  :  (0.9066677539496737, 0.880456889442105)
900  :  (0.941737540064015, 0.9399751309269405)
1000  :  (0.9506538970913907, 0.9482737577995091)
1100  :  (0.9525324603068976, 0.9498349502591947)
1200  :  (0.9548664823740152, 0.9519423706036194)
1300  :  (0.9543330227849427, 0.9522451860572073)
1400  :  (0.9583339697029863, 0.9620097021855012)
1500  :  (0.9608812665137926, 0.96411776116499)
1600  :  (0.9582164231370914, 0.9623068215861276)
1700  :  (0.9609325397258304, 0.9642769280857345)
1800  :  (0.959317979006347, 0.963134592986058)
1900  :  (0.7345743996125055, 0.6554107812450941)
2000  :  (0.91770594

(0.916870850415313, 0.9157826081584531)

In [44]:
print(dt.test_pos_edge_index, dt.test_neg_edge_index)


tensor([[ 133,   94,  167,  ..., 1094,  133,  249],
        [ 350,  142,  680,  ..., 1260, 1245,  448]]) tensor([[ 331,  263,  344,  ...,  299,    2,  424],
        [1320, 1015, 1835,  ..., 3106,  526,  965]])


In [48]:
print(dt.test_pos_edge_index.shape)
print(dt.test_neg_edge_index.shape)

torch.Size([2, 1354])
torch.Size([2, 1354])


- chercher sur differentes couches 
- tester dropout
- tester entrainer decodeur